In [ ]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, Polygon
import folium

In [ ]:
# Load Luxembourg border coordinates
lux_border_coords = pd.read_csv('lux-border-coords.csv')

# Create a polygon from the border coordinates
lux_border_polygon = Polygon(zip(lux_border_coords['lon'], lux_border_coords['lat']))

In [ ]:
# Function to check if a point is within the Luxembourg border polygon
def is_point_in_luxembourg(lat, lon):
    point = Point(lon, lat)
    return lux_border_polygon.contains(point)

In [ ]:
# Load points to check
places_df = pd.read_excel('places.xlsx')

# Check each point in the provided dataset to see if it falls within Luxembourg
places_df['within_luxembourg'] = places_df.apply(lambda row: is_point_in_luxembourg(row['latitude'], row['longitude']), axis=1)

# Filter points that are within Luxembourg
points_within_lux = places_df[places_df['within_luxembourg']]

In [ ]:
# Create an HTML map with the border of Luxembourg drawn in blue
map_places_with_border = folium.Map(location=[49.8153, 6.1296], zoom_start=10)

# Add Luxembourg border polygon to the map
folium.Polygon(locations=[(lat, lon) for lat, lon in zip(lux_border_coords['lat'], lux_border_coords['lon'])],
               color='blue', weight=2.5, opacity=1).add_to(map_places_with_border)

# Add points to the map
for idx, row in points_within_lux.iterrows():
    folium.CircleMarker(location=(row['latitude'], row['longitude']),
                        radius=2,
                        color='green',
                        fill=True).add_to(map_places_with_border)

# Save the revised map to an HTML file
map_places_with_border.save('revised_places_within_luxembourg_map.html')